# Creación de modelo con Amazon Rekognition

Este script maneja el entrenamiento de modelos de Custom Labels en Amazon Rekognition, incluyendo la creación del proyecto y monitoreo del entrenamiento.

## 1. Configuración Inicial
Importación de bibliotecas necesarias y configuración del sistema.

In [6]:
import boto3
import logging
from datetime import datetime

# Configuración básica
CONFIG = {
    "PROJECT_NAME": "ProyectoCustomLabels", # Nombre del proyecto
    "AWS_REGION": "<REGION>" # Región de AWS
}

## 2. Creación de proyecto
Crea un nuevo proyecto en Amazon Rekognition Custom Labels.

In [7]:
def create_custom_labels_project():
    try:
        # Crear cliente de Rekognition
        client = boto3.client('rekognition', region_name=CONFIG['AWS_REGION'])
        
        # Crear proyecto nuevo
        response = client.create_project(
            ProjectName=CONFIG['PROJECT_NAME']
        )
        
        # Obtener y registrar el ARN del proyecto
        project_arn = response['ProjectArn']
        logger.info(f"Proyecto creado exitosamente. ARN: {project_arn}")
        return project_arn
        
    except Exception as e:
        logger.error(f"Error al crear el proyecto: {str(e)}")
        raise

if __name__ == "__main__":
    project_arn = create_custom_labels_project()

## 3. Creación del dataset / Consola
- Acceder a Amazon Rekognition > Custom Labels
- Seleccionar el proyecto creado
- Click en "Create dataset"
- Click en "Start with a single dataset" en caso de que solo se tenga un set de imagenes, ya que Rekognition dividirá este set en 2 subsets, uno de prueba y otro de test.
- Clic en "Import images from S3 bucket"
- Especificar ubicación del bucket en la opción S3 URI
- Finalizar con "Create dataset"

## 4. Proceso de etiquetado y entrenamiento / Consola
- Iniciar con "Start labeling"
- Click en "Add labels" y agrega la etiqueta deseada por ej: Logo
- Para cada imagen:
1. Dibujar bounding box alrededor del objeto
2. Asignar etiqueta correspondiente
3. Guardar y continuar
- Click en "Save changes"
- Click en "Train model"
- Seleccionar el proyecto. Este codigo crea el proyecto llamado "MiProyectoCustomLabels"
- Click en "Train model"

## 5. Encience tu modelo
Inicia el modelo que entrenaste y dejalo listo para hacer inferencias.

In [ ]:
def start_model(project_arn, model_arn, version_name, min_inference_units):
    try:
        # Iniciar el modelo
        print('Starting model: ' + model_arn)
        response=client.start_project_version(ProjectVersionArn=model_arn, MinInferenceUnits=min_inference_units)
        # Espere a que el modelo esté en estado de ejecución
        project_version_running_waiter = client.get_waiter('project_version_running')
        project_version_running_waiter.wait(ProjectArn=project_arn, VersionNames=[version_name])

        # Obtener el estado de ejecución
        describe_response=client.describe_project_versions(ProjectArn=project_arn,
            VersionNames=[version_name])
        for model in describe_response['ProjectVersionDescriptions']:
            print("Status: " + model['Status'])
            print("Message: " + model['StatusMessage']) 
    except Exception as e:
        print(e)
        
    print('El modelo está listo...')
    
def main():
    project_arn='<ARN_PROYECTO>' # ARN del proyecto
    model_arn='<ARN_MODELO>' # ARN del modelo
    min_inference_units=1 
    version_name='ProyectoCustomLabels.2025-07-18T15.41.48' # Version del modelo
    start_model(project_arn, model_arn, version_name, min_inference_units)

if __name__ == "__main__":
    main()

## Notas Finales
- Crea y gestiona proyectos de Amazon Rekognition Custom Labels
- Implementa un sistema de creación de proyectos automatizado